In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE162253"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE162253"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE162253.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE162253.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE162253.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Bacterial effect on gene expression"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: female', 'mouse strain: C57BL/6J-APCmin/J', 'mouse strain: C57BL/6J'], 1: ['strain: C57BL/6J', 'tissue: intestine'], 2: ['tissue: bladder', 'experiment: exp1', 'experiment: exp2', 'experiment: exp3'], 3: ['experiment: exp1', 'experiment: exp2', 'experiment: exp3', 'treatment: rLon', 'treatment: 536', 'treatment: N/A', 'treatment: PBS'], 4: ['treatment: PAI1', 'treatment: 536', 'treatment: N/A', 'treatment: PBS', 'treatment: rLon', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# Define variables for dataset assessment
is_gene_available = True  # The dataset appears to be about gene expression based on the title

# For trait data (bladder cancer vs. control)
# Based on the sample characteristics, trait data is not explicitly available
# The infection status in row 1 could be used as a trait, but it's not related to bladder cancer
trait_row = None  # No direct bladder cancer data

# For age and gender data
age_row = None  # No age information available
gender_row = None  # No gender information available

# Define conversion functions (even though we won't use them in this case)
def convert_trait(value_str):
    # Not used as trait_row is None
    return None

def convert_age(value_str):
    # Not used as age_row is None
    return None

def convert_gender(value_str):
    # Not used as gender_row is None
    return None

# Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since trait_row is None, we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at',
       '1415674_PM_a_at', '1415675_PM_at', '1415676_PM_a_at', '1415677_PM_at',
       '1415678_PM_at', '1415679_PM_at', '1415680_PM_at', '1415681_PM_at',
       '1415682_PM_at', '1415683_PM_at', '1415684_PM_at', '1415685_PM_at',
       '1415686_PM_at', '1415687_PM_a_at', '1415688_PM_at', '1415689_PM_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (format: 11715100_at) rather than standard human gene symbols
# These probe IDs typically need to be mapped to human gene symbols for biological interpretation
# Standard human gene symbols would look like BRCA1, TP53, etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at', '1415674_PM_a_at'], 'GB_ACC': ['BC024686', 'NM_013477', 'NM_020585', 'NM_133900', 'NM_021789'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus'], 'Annotation Date': ['Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenBank', 'GenBank'], 'Target Description': ['gb:BC024686.1 /DB_XREF=gi:19354080 /FEA=FLmRNA /CNT=416 /TID=Mm.26422.1 /TIER=FL+Stack /STK=110 /UG=Mm.26422 /LL=54161 /UG_GENE=Copg1 /DEF=Mus musculus, coatomer protein complex, subunit gamma 1, clone MGC:30335 IMAGE:3992144, mRNA, complete cds. /PROD=coatomer protein complex, subunit gamma 1 /FL=gb:AF187079.1 gb:BC024686.1

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation DataFrame
print("Looking at the gene annotation and expression data ID formats:")
print(f"Gene annotation ID example: {gene_annotation['ID'].iloc[0]}")
print(f"Gene expression ID example: {gene_data.index[0]}")

# Check species information
print(f"Species in annotation: {gene_annotation['Species Scientific Name'].iloc[0]}")

# Given this is mouse data and not human data, and we're studying human bladder cancer,
# this dataset is not appropriate. We should set is_gene_available to False.
is_gene_available = False

# Check for matching IDs just to confirm our suspicion
common_ids = set(gene_annotation['ID'].astype(str)) & set(gene_data.index)
print(f"Number of common IDs between annotation and expression data: {len(common_ids)}")

# Since this is mouse data and not suitable for human bladder cancer study,
# we'll create an empty gene_data_mapped DataFrame to indicate no valid mapping
gene_data_mapped = pd.DataFrame()

# Print information about the result
print(f"Original gene expression data shape: {gene_data.shape}")
print(f"Dataset contains mouse gene data, not suitable for human bladder cancer study")

# Update gene_data to reflect this issue
gene_data = gene_data_mapped

# Update metadata to reflect that gene data is not available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

Looking at the gene annotation and expression data ID formats:
Gene annotation ID example: 1415670_PM_at
Gene expression ID example: 1415670_PM_at
Species in annotation: Mus musculus
Number of common IDs between annotation and expression data: 45077
Original gene expression data shape: (45077, 45)
Dataset contains mouse gene data, not suitable for human bladder cancer study


False